# Code

In [400]:
import re
from pathlib import Path
import subprocess
from tqdm import tqdm

In [320]:
def listify(o):
    """
    Convert to list
    """
    if o is None:
        return []
    if isinstance(o, list):
        return o
    if isinstance(o, str):
        return [o]
    return [o]


In [321]:
def compose(x, funcs, *args, order_key="_order", **kwargs):
    """
    Chain functions
    """
    key = lambda o: getattr(o, order_key, 0)
    for f in sorted(listify(funcs), key=key):
        x = f(x, **kwargs)
    return x


In [369]:
def replacers(x, p1, p2 = ""): return x.replace(p1, p2)
def space_remover(x): return replacers(x, " ")
def nl_remover(x): return replacers(x, "\n")
def curly_split(x): return re.findall(r"\{(.*?)\}", x, re.MULTILINE | re.DOTALL)
def between_curlies(x): return re.findall(r"\}(.*?)\{", x, re.MULTILINE | re.DOTALL)
def comma_split(x): return [y.split(",") for y in x]
def single_to_dict(x):
    tmp_dict = {}
    for i in x:
        split_string = i.split(":")
        tmp_dict[split_string[0]] = split_string[1]
    return tmp_dict
def multiple_to_dict(x): return [single_to_dict(y) for y in x]
def preprocess_input(x): return compose(x, [space_remover, nl_remover, curly_split, comma_split, multiple_to_dict])
def preprocess_merges(x): return compose(x, [space_remover, nl_remover, between_curlies])

In [417]:
def exec_ffmpeg_com(vid_path, tmp_path, comm):
    ou1 = subprocess.getoutput(f"yes|ffmpeg -i {str(vid_path)} {comm} {str(tmp_path)}")
    return ou1

In [418]:
def remove_audio(val, vid_path, tmp_path,): 
    if val ==0: exec_ffmpeg_com(vid_path, tmp_path, "-c copy -an")

In [421]:
def rescale_video(val, vid_path, tmp_path): 
    if len(val)>2: exec_ffmpeg_com(vid_path, tmp_path, f'-vf scale={";".join(val.split(";"))}')

# Test

In [422]:
main_qu = []

In [423]:
main_path = Path("."); main_path

PosixPath('.')

In [424]:
test_string = '''{
name:grassTutorialNew.mp4,
A:0,
resize:128;128
}+{name:venus.mp4,
A:0,
resize:128;128
}
'''

In [425]:
dic_procs = {
    "A": remove_audio,
    "resize": rescale_video,
}

In [443]:
def process_video(dic, fpath):
    name = dic["name"]
    vid_path = fpath/name
    tmp_path = fpath/f"tmp_{name}"
    
    print(dic.keys())
    
    proc_list = [dic_procs[key](dic[key], vid_path, tmp_path) for key in dic.keys() if key not in ["name"]]
    print(proc_list)
    
    return tmp_path

In [444]:
def run_process(string, fpath):
    pi, pm = preprocess_input(string), preprocess_merges(string)
    executed_vid_list = []
    for i,vid in tqdm(enumerate(pi)):
        process_video(vid, fpath)
        break
#     print(pi, pm, fpath)

In [445]:
run_process(test_string, main_path)

0it [00:00, ?it/s]

dict_keys(['name', 'A', 'resize'])
[None, None]
